## Homework
The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

### Q1. Downloading the data
[The data link address of NYC Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

Read the data for January. How many columns are there?

In [44]:
import pandas as pd
# This is January data of NYC Dataset
nyc_dataframe = pd.read_parquet("../data/yellow_tripdata_2022-01.parquet")

In [45]:
print("Number of columns are: ", nyc_dataframe.shape[1])

Number of columns are:  19


### Q2. Computing duration
Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.
What's the standard deviation of the trips duration in January?

Let's first check the info of the dataframe to see the type of `tpep_dropoff_datetime` and `tpep_pickup_datetime` columns

In [54]:
nyc_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

Since we will find the `duration` of the trips in minutes, the date time type is what we are looking for. We don't need to convert it to datetime type.

In [55]:
nyc_dataframe["duration"] = nyc_dataframe["tpep_dropoff_datetime"]-nyc_dataframe["tpep_pickup_datetime"]
nyc_dataframe.duration = nyc_dataframe.duration.apply(lambda td: td.total_seconds()/60)

In [58]:
import math
"""
The calculation of standard deviation is described as;
std = sqrt(mean(x)) ,
where x = abs(a - a.mean())**2 . The average squared deviation is typically calculated as x.sum() / N , where N = len(x)
"""

# according to that calculation description we can define our solution like below
x = abs(nyc_dataframe.duration-nyc_dataframe.duration.mean())**2
print(math.sqrt(x.sum() / len(x)))

# we can also choose the easy way and use built-in function
print(nyc_dataframe.duration.std())

# 46.45 is the rigth answer for this question

46.445295712725304
46.44530513776802


### Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?